工作平台环境报告

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import platform
import locale
import json
import site
import subprocess
from datetime import datetime
from pathlib import Path

def run_cmd(cmd):
    try:
        r = subprocess.run(cmd, capture_output=True, text=True, check=False)
        out = (r.stdout or "").strip()
        err = (r.stderr or "").strip()
        if out and err:
            return out + "\n" + err
        return out or err
    except Exception as e:
        return f"<failed: {e}>"

def bytes_to_human(n):
    if n is None:
        return "N/A"
    step = 1024.0
    for unit in ["B", "KB", "MB", "GB", "TB", "PB"]:
        if n < step:
            return f"{n:.2f} {unit}"
        n /= step
    return f"{n:.2f} EB"

def safe_get_env(keys):
    d = {}
    for k in keys:
        v = os.environ.get(k)
        if v is not None:
            d[k] = v
    return d

def print_section(title):
    print("\n" + "=" * 80)
    print(title)
    print("=" * 80)

def main():
    print_section("Timestamp")
    print(datetime.now().isoformat())

    print_section("OS / Platform")
    print("platform:", platform.platform())
    print("system  :", platform.system())
    print("release :", platform.release())
    print("version :", platform.version())
    print("machine :", platform.machine())
    print("processor:", platform.processor())
    print("uname   :", platform.uname())

    print_section("Python")
    print("executable :", sys.executable)
    print("version    :", sys.version.replace("\n", " "))
    print("python_ver :", platform.python_version())
    print("implementation:", platform.python_implementation())
    print("prefix     :", sys.prefix)
    print("base_prefix:", getattr(sys, "base_prefix", None))
    print("cwd        :", os.getcwd())
    print("argv       :", sys.argv)

    print_section("Locale / Encoding / Timezone")
    print("preferred encoding:", locale.getpreferredencoding(False))
    print("filesystem encoding:", sys.getfilesystemencoding())
    print("default locale:", locale.getdefaultlocale())
    print("TZ env:", os.environ.get("TZ"))

    print_section("CPU / Cores")
    print("os.cpu_count:", os.cpu_count())
    # Linux: /proc/cpuinfo quick peek
    if Path("/proc/cpuinfo").exists():
        try:
            txt = Path("/proc/cpuinfo").read_text(errors="ignore")
            model_lines = [line for line in txt.splitlines() if "model name" in line]
            if model_lines:
                print("cpu model (first):", model_lines[0].split(":", 1)[-1].strip())
        except Exception as e:
            print("cpuinfo:", e)

    print_section("Memory (best-effort)")
    # Linux: /proc/meminfo
    if Path("/proc/meminfo").exists():
        try:
            meminfo = Path("/proc/meminfo").read_text(errors="ignore").splitlines()
            kv = {}
            for line in meminfo:
                if ":" in line:
                    k, v = line.split(":", 1)
                    kv[k.strip()] = v.strip()
            print("MemTotal:", kv.get("MemTotal"))
            print("MemFree :", kv.get("MemFree"))
            print("MemAvailable:", kv.get("MemAvailable"))
        except Exception as e:
            print("meminfo:", e)
    else:
        print("No /proc/meminfo (non-Linux or restricted environment)")

    print_section("Disk")
    try:
        import shutil
        total, used, free = shutil.disk_usage(os.getcwd())
        print("disk usage for cwd:")
        print("  total:", bytes_to_human(total))
        print("  used :", bytes_to_human(used))
        print("  free :", bytes_to_human(free))
    except Exception as e:
        print("disk_usage:", e)

    print_section("Network (best-effort)")
    print("hostname:", platform.node())
    # Try ip/ifconfig
    if platform.system().lower() == "linux":
        print("ip addr:\n", run_cmd(["ip", "addr"]))
        print("ip route:\n", run_cmd(["ip", "route"]))
    elif platform.system().lower() == "darwin":
        print("ifconfig:\n", run_cmd(["ifconfig"]))
        print("netstat -rn:\n", run_cmd(["netstat", "-rn"]))
    elif platform.system().lower() == "windows":
        print("ipconfig:\n", run_cmd(["ipconfig", "/all"]))

    print_section("Environment Variables (common)")
    common_keys = [
        "PATH", "PYTHONPATH", "VIRTUAL_ENV", "CONDA_PREFIX", "CONDA_DEFAULT_ENV",
        "PIP_INDEX_URL", "PIP_EXTRA_INDEX_URL", "HTTP_PROXY", "HTTPS_PROXY", "NO_PROXY",
        "CUDA_VISIBLE_DEVICES", "LD_LIBRARY_PATH", "DYLD_LIBRARY_PATH",
    ]
    print(json.dumps(safe_get_env(common_keys), ensure_ascii=False, indent=2))

    print_section("site / sys.path")
    try:
        print("site.getsitepackages:", getattr(site, "getsitepackages", lambda: "<n/a>")())
    except Exception as e:
        print("site.getsitepackages error:", e)
    try:
        print("site.getusersitepackages:", site.getusersitepackages())
    except Exception as e:
        print("site.getusersitepackages error:", e)

    print("\n--- sys.path ---")
    for p in sys.path:
        print(p)

    print_section("Installed Packages (pip)")
    print(run_cmd([sys.executable, "-m", "pip", "--version"]))
    print(run_cmd([sys.executable, "-m", "pip", "list", "--format=columns"]))

    print_section("Optional: GPU (best-effort)")
    # nvidia-smi if exists
    print("nvidia-smi:\n", run_cmd(["nvidia-smi"]))
    # torch cuda info if available
    try:
        import torch
        print("torch:", torch.__version__)
        print("torch.cuda.is_available:", torch.cuda.is_available())
        if torch.cuda.is_available():
            print("torch.cuda.device_count:", torch.cuda.device_count())
            print("torch.cuda.get_device_name(0):", torch.cuda.get_device_name(0))
    except Exception as e:
        print("torch not available or failed:", e)

if __name__ == "__main__":
    main()


Timestamp
2026-01-11T23:51:50.628433

OS / Platform
platform: macOS-13.5-arm64-arm-64bit-Mach-O
system  : Darwin
release : 22.6.0
version : Darwin Kernel Version 22.6.0: Wed Jul  5 22:21:53 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T6020
machine : arm64
processor: arm
uname   : uname_result(system='Darwin', node='anonymousdeMacBook-Pro.local', release='22.6.0', version='Darwin Kernel Version 22.6.0: Wed Jul  5 22:21:53 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T6020', machine='arm64')

Python
executable : /opt/anaconda3/envs/deep-neural-networks-practise/bin/python
version    : 3.13.11 | packaged by Anaconda, Inc. | (main, Dec 10 2025, 21:21:08) [Clang 20.1.8 ]
python_ver : 3.13.11
implementation: CPython
prefix     : /opt/anaconda3/envs/deep-neural-networks-practise
base_prefix: /opt/anaconda3/envs/deep-neural-networks-practise
cwd        : /Volumes/cloud/learn/deep-neural-networks-practise/workspace
argv       : ['/opt/anaconda3/envs/deep-neural-networks-practise/lib/python3

/var/folders/3h/ryb34lc91xl49grgtmybgktr0000gn/T/ipykernel_33510/2652973984.py:74: DeprecationWarning: 'locale.getdefaultlocale' is deprecated and slated for removal in Python 3.15. Use setlocale(), getencoding() and getlocale() instead.
  print("default locale:", locale.getdefaultlocale())


pip 25.3 from /opt/anaconda3/envs/deep-neural-networks-practise/lib/python3.13/site-packages/pip (python 3.13)
Package                 Version
----------------------- -----------
absl-py                 2.3.1
astunparse              1.6.3
attrs                   25.4.0
certifi                 2026.1.4
charset-normalizer      3.4.4
cloudpickle             3.1.2
contourpy               1.3.3
cycler                  0.12.1
decorator               5.2.1
DeepXDE                 1.15.0
dm-tree                 0.1.9
filelock                3.20.3
flatbuffers             25.12.19
fonttools               4.61.1
fsspec                  2026.1.0
gast                    0.7.0
google-pasta            0.2.0
grpcio                  1.76.0
h5py                    3.15.1
idna                    3.11
Jinja2                  3.1.6
joblib                  1.5.3
keras                   3.13.0
kiwisolver              1.4.9
libclang                18.1.1
Markdown                3.10
markdown-it-py          4

创建 conda 环境
```shell
conda create -n deep-neural-networks-practise python=3.13
```

激活 conda 环境
```shell
conda activate conda deep-neural-networks-practise
```

安装依赖项
```shell
conda env create -f environment.yml
```

实验后导出更新 conda 环境
```shell
bash export_requirements.sh
```